In [ ]:
import pandas as pd
import os
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc


### Single year (2023) explorations

In [ ]:
df = pd.read_csv('datafiles/combined_df_2023.csv', index_col = 0)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
ladder_sorted_df = df.sort_values(by=['Ladder score'], ascending=False)

In [ ]:
top_10_countries = ladder_sorted_df.head(10)
bottom_10_countries = ladder_sorted_df.tail(10)

In [ ]:
bottom_10_countries

In [ ]:
grouped_regions = df.groupby(by=['Regional Indicator'], as_index=False)

In [ ]:
from pandas.api.types import is_numeric_dtype

In [ ]:
numeric_cols = []
non_numeric_cols = []

In [ ]:
for col in df.columns:
    if is_numeric_dtype(df[col]):
        numeric_cols.append(col)
    else:
        non_numeric_cols.append(col)    

In [ ]:
numeric_cols

In [ ]:
grouped_regions.dtypes

In [ ]:
grouped_regions_mean = grouped_regions[[*numeric_cols]].mean()

In [ ]:
grouped_regions_mean.sort_values(by="Ladder score", inplace=True, ascending=False)

In [ ]:
grouped_regions_mean

In [ ]:
#ff7f0e

In [ ]:
# ladder_gdp_fig = px.bar(grouped_regions_mean, y='Ladder score', x='Regional Indicator')
# ladder_gdp_fig.add_trace(px.scatter(grouped_regions_mean, x='Regional Indicator', y='Logged GDP per capita',
#                                 line=dict(color='#ff7f0e')).data[0])

In [ ]:
ladder_gdp_fig = px.bar(grouped_regions_mean, y='Ladder score', x='Regional Indicator')
ladder_gdp_fig.add_trace(go.Scatter(x=grouped_regions_mean['Regional Indicator'],
                                    y=grouped_regions_mean['Logged GDP per capita'],
                                    line=dict(color='#ff7f0e'),
                                    name='GDP',
                                    mode='lines'))

### ========== Dash app ==========

In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

#### Metric vs ladder score

In [ ]:
metric_vs_ladder_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='metric-vs-ladder-selection',
                value = 'Social support',
                options = list(df.columns)
            )
            ]),
    ],
    body=True,
)

In [ ]:
metric_vs_ladder_container = dbc.Container([
    html.H1(children = 'Chosen metric vs ladder score', style={'textAlign': 'center'}),
    html.P(id='metric-vs-ladder-test'),
    dbc.Row(
        [
            dbc.Col(metric_vs_ladder_controls, md=4),
            dbc.Col(dcc.Graph(id='metric-vs-ladder-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('metric-vs-ladder-graph', 'figure'),
        Output('metric-vs-ladder-test', 'children')
    ],
    [
        Input('metric-vs-ladder-selection', 'value'),
    ]
)
def update_metric_vs_ladder_graph(metric):
    fig = px.scatter(df, y='Ladder score', x=metric, hover_data='Country', color='Regional Indicator',
                    trendline='ols')
    return fig, f'metric: {metric}'

#### Metric vs life expectancy

In [ ]:
metric_vs_life_controls = dbc.Card(
    [
        html.Div(
            [
            dbc.Label("Metric"),
            dcc.Dropdown(
                id='metric-vs-life-selection',
                value = 'Social support',
                options = list(df.columns)
            )
            ]),
    ],
    body=True,
)

In [ ]:
metric_vs_life_container = dbc.Container([
    html.H1(children = 'Chosen metric vs life expectancy', style={'textAlign': 'center'}),
    html.P(id='metric-vs-life-test'),
    dbc.Row(
        [
            dbc.Col(metric_vs_life_controls, md=4),
            dbc.Col(dcc.Graph(id='metric-vs-life-graph'), md=8),
        ],
        align="center",
    ),
], fluid=True)

In [ ]:
@app.callback(
    [
        Output('metric-vs-life-graph', 'figure')
        ,Output('metric-vs-life-test', 'children')
    ],
    [
        Input('metric-vs-life-selection', 'value'),
    ]
)
def update_metric_vs_ladder_graph(metric):
    fig = px.scatter(df, y='Life expectancy', x=metric, hover_data='Country', color='Regional Indicator')
    return fig, f'metric: {metric}'

In [ ]:
# px.scatter(df, y='Ladder score', x='Social support', hover_data='Country', color='Regional Indicator', trendline='ols')

In [ ]:
app.layout = dbc.Container([
    metric_vs_ladder_container,
    html.Hr(),
    metric_vs_life_container,
    html.Hr(),
], fluid=True)

In [ ]:
app.run(debug=True)

In [ ]:
px.scatter(df, x='Infant mortality', y='Physicians per thousand')

### ========== tests ===========

In [ ]:
fig = px.scatter(df, x="Logged GDP per capita", y="Life expectancy", size="Social support",
                 color="Regional Indicator", hover_data='Country')

In [ ]:
fig.show()